In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import pickle
from tqdm.notebook import tqdm as tqdm_n
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
import gc
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from tensorflow.python.keras import backend as K
from loop import TrainingLoop

In [2]:
data_path = 'data'
train_data = np.load(os.path.join(data_path, 'twitter_train_vectors.npy'), allow_pickle=True)
test_data = np.load(os.path.join(data_path, 'twitter_test_vectors.npy'), allow_pickle=True)
train_labels = np.load(os.path.join(data_path, 'twitter_train_labels.npy'), allow_pickle=True)
test_labels = np.load(os.path.join(data_path, 'twitter_test_labels.npy'), allow_pickle=True)

In [3]:
def average_on_window(data, label, size):
    new_data = []
    new_labels = []
    for i in tqdm_n(range(len(data))):
        sample = data[i]
        if len(sample) >= size:
            new_sample = []
            count = int(np.floor((len(sample) / size)))
            rest = int(len(sample) / size)
            for j in range(size-1):
                new_sample.append(np.array(sample[j*count:(j+1)*count].mean(axis=0)))
            if rest != 0:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count+rest].mean(axis=0)))
            else:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count].mean(axis=0)))
            new_data.append(np.array(new_sample))
            new_labels.append(label[i])
    return np.array(new_data), np.array(new_labels)

In [4]:
X_train, y_train = average_on_window(train_data, train_labels, 2)
X_test, y_test = average_on_window(test_data, test_labels, 2)

In [5]:
tf.random.set_seed(42)
np.random.seed(42)
model = Sequential()

model.add(LSTM(5, return_sequences = True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='selu'))
model.add(Dense(50, activation='selu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [6]:
optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.BinaryCrossentropy()

batch_size = 1024
epochs = 50

In [7]:
train_metrics = tf.keras.metrics.BinaryAccuracy()
val_metrics = tf.keras.metrics.BinaryAccuracy()

In [11]:
@tf.function
def calc_loss(x_train, y_train):
    with tf.GradientTape() as tape:
        logits = model(x_train, training=False)
        loss_value = loss_function(y_train, logits)
    return loss_value

In [13]:
def batch_selector(data, idx):
    largest_loss = 0
    lergest_loss_idx = idx
    for i in range(idx, idx+5):
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        loss = calc_loss(x_batch_train, y_batch_train)
        if loss > largest_loss:
            largest_loss = loss
            largest_loss_idx = i
    return largest_loss_idx, largest_loss

In [14]:
training = TrainingLoop(model, X_train, y_train, loss_function, optimizer, train_metrics, val_metrics, 
                        validation_split=0.1, batch_selection=batch_selector, batch_size=batch_size)
training.train(epochs)

	  0% | 0/94 [00:00<?, ?it/s]


ValueError: in user code:

    <ipython-input-11-ee995b19888e>:4 calc_loss  *
        logits = model(x_train, training=False)
    /home/imetomi/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:975 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs,
    /home/imetomi/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/input_spec.py:176 assert_input_compatibility
        raise ValueError('Input ' + str(input_index) + ' of layer ' +

    ValueError: Input 0 of layer sequential is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: [250]


In [ ]:
model.evaluate(X_test, y_test, batch_size=batch_size)